# #-------------------------- Goal --------------------------------#

1) Build the generator and discriminator components of a GAN from scratch.
2) Create generator and discriminator loss functions.
3) Train your GAN and visualize the generated images.


In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchsummary import summary
torch.manual_seed(0) # Set for testing purposes

ModuleNotFoundError: No module named 'torch'

In [ ]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

# Batches
<li>While you could train your model after generating one image, it is extremely inefficient and leads to less stable training. In GANs, and in machine learning in general, you will process multiple images per training step. These are called batches.
</li>
<li>
This means that your generator will generate an entire batch of images and receive the discriminator's feedback on each before updating the model. The same goes for the discriminator, it will calculate its loss on the entire batch of generated images as well as on the reals before the model is updated.
</li>


# Generator
<li> The first step is to build the generator component.</li>
<li>
we will start by creating a function to make a single layer/block for the generator's neural network. Each block should include a linear transformation to map to another shape, a batch normalization for stabilization, and finally a non-linear activation function (you use a ReLU here) so the output can be transformed in complex ways.</li>

In [ ]:
def get_generator_block(input_dim, output_dim):
    """
    When inplace=True is passed to nn.LeakyReLU(), it means that the operation should be performed "in place", 
    which means that the input tensor is modified in place without creating a new tensor.
    """
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True)
    )

In [ ]:
# Verify the generator block function
def test_gen_block(in_features, out_features, num_test=1000):
    block = get_generator_block(in_features, out_features)

    # Check the three parts
    assert len(block) == 3
    assert type(block[0]) == nn.Linear
    assert type(block[1]) == nn.BatchNorm1d
    assert type(block[2]) == nn.ReLU
    
    # Check the output shape
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    assert tuple(test_output.shape) == (num_test, out_features)
    assert test_output.std() > 0.55
    assert test_output.std() < 0.65

test_gen_block(25, 12)
test_gen_block(15, 28)
print("Success!")

# Now Lets Create Generator Class
<li>Takes the random noise and generate image with the Proper Image dimension

In [ ]:
class Generator(nn.Module):

    def __init__(self, noise_dimension=10, hidden_units=64, image_size=28*28):
        super(Generator, self).__init__()
        
        self.gen = nn.Sequential(

            # First layer Takes the noise as the input with 64 neurons/hidden units
            get_generator_block(input_dim=noise_dimension, output_dim=hidden_units),
            # 2nd layer
            get_generator_block(input_dim=hidden_units, output_dim=hidden_units*2),
            # 3rd layer
            get_generator_block(input_dim=hidden_units*2, output_dim=hidden_units*2),
            # 4th layer 
            get_generator_block(input_dim=hidden_units*2, output_dim=hidden_units*2),
            # 5th layer
            get_generator_block(input_dim=hidden_units*2, output_dim=image_size),
            # sigmoid activation bcz the output of the above layer is flattended vector with elements value ranges from - to + we will make them in between 0 and 1
            nn.Sigmoid()
            )
        

    def forward(self, noise):
            # noise: a noise tensor with dimensions(num_samples,noise_dimension)
            return self.gen(noise)
        
    def get_gen(self):
            return  self.gen

In [ ]:
test_gen = Generator().get_gen()

In [ ]:
test_gen

In [ ]:
test_gen.__getitem__(4)[0]

In [ ]:
# Verify the generator class
def test_generator(noise_dim, image_dim, units, num_test=10000):
    gen = Generator(noise_dimension=noise_dim,hidden_units=units, image_size=image_dim).get_gen()
    
    # Check there are six modules in the sequential part
    assert len(gen) == 6
    assert str(gen.__getitem__(4)[0]).replace(' ', '') == f'Linear(in_features={units * 2},out_features={image_dim},bias=True)'
    assert str(gen.__getitem__(5)).replace(' ', '') == 'Sigmoid()'
    test_input = torch.randn(num_test, noise_dim)
    test_output = gen(test_input)

    # Check that the output shape is correct
    assert tuple(test_output.shape) == (num_test, image_dim)
    assert test_output.max() < 1, "Make sure to use a sigmoid"
    assert test_output.min() > 0, "Make sure to use a sigmoid"
    assert test_output.std() > 0.05, "Don't use batchnorm here"
    assert test_output.std() < 0.15, "Don't use batchnorm here"

test_generator(10, 20, 10)
test_generator(20, 8, 24)
print("Success!")

# Noise
<li>To be able to use your generator, you will need to be able to create noise vectors. The noise vector z has the important role of making sure the images generated from the same class don't all look the same -- think of it as a random seed. You will generate it randomly using PyTorch by sampling random numbers from the normal distribution. Since multiple images will be processed per pass, you will generate all the noise vectors at once.

<li>Note that whenever you create a new tensor using torch.ones, torch.zeros, or torch.randn, you either need to create it on the target device, e.g. torch.ones(3, 3, device=device), or move it onto the target device using torch.ones(3, 3).to(device). You do not need to do this if you're creating a tensor by manipulating another tensor or by using a variation that defaults the device to the input, such as torch.ones_like. In general, use torch.ones_like and torch.zeros_like instead of torch.ones or torch.zeros where possible.

In [ ]:
def generate_noise(num_samples, noise_dimension,device="cpu"):
    """
    randn:
    Returns a tensor filled with random numbers from a normal distribution
    with mean `0` and variance `1` (also called the standard normal
    distribution)
    """
    return torch.randn(num_samples,noise_dimension,device=device)

In [ ]:
# Verify the noise vector function
def test_get_noise(n_samples, z_dim, device='cpu'):
    noise = generate_noise(n_samples, z_dim, device)
    
    # Make sure a normal distribution was used
    assert tuple(noise.shape) == (n_samples, z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device).startswith(device)

test_get_noise(1000, 100, device='cpu')
if torch.cuda.is_available():
    test_get_noise(1000, 32, 'cuda')
print("Success!")

# Discriminator
<li>The second component that you need to construct is the discriminator. As with the generator component, you will start by creating a function that builds a neural network block for the discriminator.

<li>Note: You use leaky ReLUs to prevent the "dying ReLU" problem, which refers to the phenomenon where the parameters stop changing due to consistently negative values passed to a ReLU, which result in a zero gradient. 

In [ ]:
def get_discriminator_block(input_dim, output_dim):

    """
    Instead of setting negative values to zero, Leaky ReLU sets negative values to a small negative slope (usually 0.2 or 0.01).
    """

    return nn.Sequential(
        nn.Linear(in_features=input_dim, out_features=output_dim),
        nn.LeakyReLU(0.2, inplace=True)
    )

In [ ]:
# Verify the discriminator block function
def test_disc_block(in_features, out_features, num_test=10000):
    block = get_discriminator_block(in_features, out_features)

    # Check there are two parts
    assert len(block) == 2
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)

    # Check that the shape is right
    assert tuple(test_output.shape) == (num_test, out_features)
    
    # Check that the LeakyReLU slope is about 0.2
    assert -test_output.min() / test_output.max() > 0.1
    assert -test_output.min() / test_output.max() < 0.3
    assert test_output.std() > 0.3
    assert test_output.std() < 0.5
    
    assert str(block.__getitem__(0)).replace(' ', '') == f'Linear(in_features={in_features},out_features={out_features},bias=True)'        
    assert str(block.__getitem__(1)).replace(' ', '').replace(',inplace=True', '') == 'LeakyReLU(negative_slope=0.2)'


test_disc_block(25, 12)
test_disc_block(15, 28)
print("Success!")

Now we use these blocks to make a discriminator! The discriminator class holds 2 values:

1. The image dimension
2. The hidden dimension
<li>The discriminator will build a neural network with 4 layers. It will start with the image tensor and transform it until it returns a single number (1-dimension tensor) output. This output classifies whether an image is fake or real. Note that we do not need a sigmoid after the output layer since it is included in the loss function. Finally, to use your discrimator's neural network we are given a forward pass function that takes in an image tensor to be classified.

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, image_size=28*28, hidden_units=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # linear layer -1
            get_discriminator_block(input_dim=image_size, output_dim=hidden_units),
            # linear layer -2
            get_discriminator_block(input_dim=hidden_units, output_dim=hidden_units*2),
            # linear layer -3
            get_discriminator_block(input_dim=hidden_units*2,output_dim=hidden_units*2),
            # linear layer - 4 i.e output layer
            nn.Linear(in_features=hidden_units*2, out_features=1)

        )

    def forward(self,image):
        return self.disc(image)
    
    #getter
    def get_disc(self):
        return self.disc
    


In [ ]:
test_disc = Discriminator().get_disc()

In [ ]:
test_disc.__getitem__

In [ ]:
# Verify the discriminator class
def test_discriminator(z_dim, hidden_dim, num_test=100):
    
    disc = Discriminator(z_dim, hidden_dim).get_disc()

    # Check there are three parts
    assert len(disc) == 4
    assert type(disc.__getitem__(3)) == nn.Linear

    # Check the linear layer is correct
    test_input = torch.randn(num_test, z_dim)
    test_output = disc(test_input)
    assert tuple(test_output.shape) == (num_test, 1)

test_discriminator(5, 10)
test_discriminator(20, 8)
print("Success!")

<h1 style="text-align:center;background-color:DodgerBlue;color:white;"> Training </h1>
<h3> Now you can put it all together! First, we will set our parameters:</h3>

<ul>
<li>criterion: the loss function
<li>n_epochs: the number of times you iterate through the entire dataset when training
<li>noise_dim: the dimension of the noise vector
<li>display_step: how often to display/visualize the images
<li>batch_size: the number of images per forward/backward pass
<li>lr: the learning rate
<li>device: the device type, here using a GPU (which runs CUDA), not CPU
<li>Next, we load the MNIST dataset as tensors using a dataloader.
</ul>

<h1 style="color:DodgerBlue">Hyper Parameters </h1>

In [ ]:
criterion = nn.CrossEntropyLoss()
n_epochs = 10
noise_dim = 64
display_step = 500
batch_size = 16
lr = 0.00001
device = "cuda"

In [ ]:
# Load MNIST dataset as tensors
dataloader = DataLoader(
    MNIST('data/', download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

In [ ]:
dataloader.batch_size

<h2 style="color:DodgerBlue;">Now lets intialize Generator and Discriminator along with optimizer by passing model parameters

In [ ]:
gen = Generator(noise_dimension=noise_dim).to(device=device)
gen_opt = torch.optim.Adam(gen.parameters(),lr=lr)
disc = Discriminator().to(device=device)
disc_opt = torch.optim.Adam(disc.parameters(),lr=lr)

Before you train your GAN, you will need to create functions to calculate the discriminator's loss and the generator's loss. This is how the discriminator and generator will know how they are doing and improve themselves. Since the generator is needed when calculating the discriminator's loss, you will need to call .detach() on the generator result to ensure that only the discriminator is updated!

In [ ]:
def get_disc_loss(num_samples, noise_dim, device, real_images, criterion, generator, discriminator):

    """ 
    Step 1 : Create Noise using generate_noise function.
    Step 2 : generate images from generator by passing noise as input, i.e gen which is intialized.
    step 3 : Pass the images generated to the discriminator i.e disc which is intialized, which predict the images fake or real.
            Note : Pass the images with detach mode, bcz we are computing here disc loss
    step 4 : Compute the loss of discriminator, for the fake images.
    step 5 : Pass the real images over the discriminator and get the prediction
    step 6 : Compute the loss of discriminator for real images
    step 7 : Average the loss for real and fake images
    """

    noise = generate_noise(num_samples=num_samples, noise_dimension=noise_dim, device=device)
    fake_images = generator(noise)
    disc_fake_images_pred = discriminator(fake_images.detach())
    disc_fake_images_loss = criterion(disc_fake_images_pred, torch.zeros_like(disc_fake_images_pred))
    disc_real_images_pred = discriminator(real_images)
    disc_real_images_loss = criterion(disc_real_images_pred, torch.ones_like(disc_real_images_pred))
    disc_loss = (disc_fake_images_loss + disc_real_images_loss) / 2

    return  disc_loss

In [ ]:
# Generator loss function

def get_gen_loss(num_samples, noise_dim, device, generator, discriminator, criterion):

    """ 
    step 1 : Generate Noise
    step 2 : Generate the fake images using noise over the generator
    step 3 : pass the fake images over the discriminator
    step 4 : compute the loss, make sure that the loss is computed over real labels bcz the generator want to fool the discriminator
    """
    noise = generate_noise(num_samples=num_samples, noise_dimension=noise_dim,device=device)
    fake_images = generator(noise)
    disc_fake_images_pred = discriminator(fake_images)
    disc_fake_images_loss = criterion(disc_fake_images_pred, torch.ones_like(disc_fake_images_pred))

    return disc_fake_images_loss


In [ ]:
mean_discriminator_loss =0
mean_generator_loss = 0
cur_step = 0
for epoch in range(n_epochs):

    for real_image,_  in tqdm(dataloader):

        cur_batch_size = len(real_image)
        
        """
        Step 1 : Faltten the image
        step 2 : compute the disriminator loss
        step 3 : compute gradients w.r.t disc_loss
        step 4 : update parameters of discriminator
        step 5 : compute generator loss
        step 6 : compute gradients w.r.t gen_loss
        step 7 : update the parameters of generator
        step 8 : 
+        
        """
        real_image = real_image.view(cur_batch_size,-1).to(device)

        # Before computing discriminator loss make sure set disc_opt.zero_grad()
        disc_opt.zero_grad()

        # caluclate discriminator loss
        disc_loss = get_disc_loss(num_samples=cur_batch_size, noise_dim=noise_dim, device=device, real_images=real_image, criterion=criterion, generator=gen, discriminator=disc)

        # compute grads
        disc_loss.backward(retain_graph=True)

        # update parameters of discriminator
        disc_opt.step()

        # Before computing generator loss make sure set gen_opt.zero_grad()
        gen_opt.zero_grad()

        #  Caluclate generator loss
        gen_loss = get_gen_loss(num_samples=cur_batch_size, noise_dim=noise_dim, device=device, generator=gen, discriminator=disc, criterion=criterion)

        # compute grads no need of retain_graph
        gen_loss.backward()

        # update the parameters of generator
        gen_opt.step()

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = generate_noise(num_samples=cur_batch_size, noise_dimension=noise_dim, device=device)
            fake = gen(fake_noise)
            show_tensor_images(fake)
            show_tensor_images(real_image)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1
